In [15]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.0/255.0 kB 5.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [7]:
!pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable
  Using cached gradio-3.37.0-py3-none-any.whl (19.8 MB)
  Using cached aiofiles-23.1.0-py3-none-any.whl (14 kB)
  Using cached altair-5.0.1-py3-none-any.whl (471 kB)
  Using cached fastapi-0.100.0-py3-none-any.whl (65 kB)
  Using cached ffmpy-0.3.1.tar.gz (5.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached gradio_client-0.2.10-py3-none-any.whl (288 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.2 MB/s eta 0:00:00
  Using cached mdit_py_plugins-0.3.3-py3-none-any.whl (50 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 7.3 MB/s eta 0:00:00
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached python_multipart-0.0.6-py3-none-any.whl (45 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached uvicorn-0.23.0-py3-none-any.whl (59 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 10.2 MB/s eta 0:00:00
     ━━

In [16]:
from llama_index import StorageContext, ServiceContext, GPTVectorStoreIndex, LLMPredictor, PromptHelper, SimpleDirectoryReader, load_index_from_storage
from langchain.chat_models import ChatOpenAI
import gradio as gr
import sys
import os
import tqdm as notebook_tqdm
from dotenv import load_dotenv
import PyPDF2

In [9]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')


In [18]:
def create_service_context():
    max_input_size = 4096
    num_outputs = 512
    max_chunk_overlap = .2
    chunk_size_limit = 600

    #allows the user to explicitly set certain constraint parameters
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    #LLMPredictor is a wrapper class around LangChain's LLMChain that allows easy integration into LlamaIndex
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0.1, model_name="gpt-4", max_tokens=num_outputs))

    #constructs service_context
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
    return service_context

def data_ingestion_indexing(directory_path):
    documents = SimpleDirectoryReader(directory_path).load_data()
    index = GPTVectorStoreIndex.from_documents(
        documents, service_context=create_service_context()
    )
    #persist index to disk, default "storage" folder
    index.storage_context.persist()

    return index

In [12]:
def data_querying(input_text):

    #rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="./storage")

    #loads index from storage
    index = load_index_from_storage(storage_context, service_context=create_service_context())
    
    #queries the index with the input text
    response = index.as_query_engine().query(input_text)
    
    return response.response

In [13]:
iface = gr.Interface(fn=data_querying,
                     inputs=gr.components.Textbox(lines=7, label="Enter your text"),
                     outputs="text",
                     title="Custom-trained Bio Knowledge Base")

In [24]:
index = data_ingestion_indexing("/Users/derekdeming/cs_projects/bioML/bio_papers/")


RetryError: RetryError[<Future at 0x292228250 state=finished raised AuthenticationError>]

In [25]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://10ccf96ceb8c86143e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
